In [1]:
import sys
sys.path.insert(0, './pypokerengine/api/')
import game
import time
setup_config = game.setup_config
start_poker = game.start_poker
import time
from randomplayer import RandomPlayer
from raise_player import RaisedPlayer
from AnotherPlayer import MyPlayer
from bot1 import Bot1

def Modifiedtestperf(agent_name1, agent1, agent_name2, agent2, num_game = 10, max_round = 1):

    # Init to play 500 games of 1000 rounds
    initial_stack = 1000
    smallblind_amount = 10

    # Init pot of players
    agent1_pot = 0
    agent2_pot = 0

    # Setting configuration
    config = setup_config(max_round=max_round, initial_stack=initial_stack, small_blind_amount=smallblind_amount)

    # Register players
    config.register_player(name=agent_name1, algorithm=agent1)
    config.register_player(name=agent_name2, algorithm=agent2)


    # Start playing num_game games
    for game in range(1, num_game+1):
        print("Game number: ", game)
        game_result = start_poker(config, verbose=0)
        agent1_pot = agent1_pot + game_result['players'][0]['stack']
        agent2_pot = agent2_pot + game_result['players'][1]['stack']

#     print("\n After playing {} games of {} rounds, the results are: ".format(num_game, max_round))
#     print("\n " + agent_name1 + "'s final pot: ", agent1_pot)
#     print("\n " + agent_name2 + "'s final pot: ", agent2_pot)

#     if (agent1_pot<agent2_pot):
#         print("\n Congratulations! " + agent_name2 + " has won.")
#     elif(agent1_pot>agent2_pot):
#         print("\n Congratulations! " + agent_name1 + " has won.")
#         # print("\n Random Player has won!")
#     else:
#         print("\n It's a draw!") 
    return (agent1_pot - agent2_pot)/(2*num_game)

In [2]:
N_POPULATION = 20
N_CROSS_OVER = 5
N_MUTATE = 3
N_ITER = 30

In [3]:
import numpy as np

In [4]:
class Gene():
        
    def __init__(self,gene):
        if gene == None:
            gene = Gene.initialize_random_gene()
        self.gene = gene
        self.fit_ness = 0
#         self.fitness = self.testPerformance()
    
    @classmethod
    def initialize_random_gene(self):
        weight = {}
        weight['opp_feature'] = np.random.normal(0,0.3,size =(3,3))
        weight['opp_bias'] = np.random.normal(0,0.3)
        return weight
        
    def readGeneFromFile(self,path):
        self.gene = cPickle.load(open(path,'rb'))
    
    def saveGeneToFile(self):
        cPickle.dump(self.gene,open('Save/Gene_%.4f'%(self.fit_ness),'wb'))
        
    def makePlayerFromGene(self):
        return MyPlayer(weight =self.gene)
    
    @classmethod
    def CrossOver(self,gene1,gene2):
        child = Gene(None)
        for i in range(2):
            child.gene[i] = np.random.choice([gene1.gene[i],gene2.gene[i]])
        return child
    
    @classmethod
    def Mutate(self,gene):
        child = Gene(None)
        for i in range(2):
            child.gene[i] = gene.gene[i] + np.random.normal(0,1) 
        return child
    
    def testPerformance(self):
        Agent = self.makePlayerFromGene()
        scoreVsRaise = Modifiedtestperf("Group 48", Agent, "RaisePlayer", Bot1(),10,10)
        scoreVsRandom = Modifiedtestperf("Group 48", Agent, "Bot1", Bot1(),10,100)
        print(scoreVsRaise, scoreVsRandom)
        return scoreVsRaise*0.2 + scoreVsRandom*0.8

In [5]:
# G = Gene(None)
# s = time.time()
# G.testPerformance()
# print(time.time()-s)

In [6]:
x =np.array([2,3,4,5,6,7,8,9,10])

def generate_game(population, n_sample):
    length = len(population)
    gamelst = []
    for i in range(length):
        prob = np.zeros(length) + float(1)/(length-1)
        prob[i] = 0
        tmp = [population[i]]
        tmp.extend(np.random.choice(population,size = n_sample, p=prob, replace = False).tolist())
        gamelst.append(tmp)
    return gamelst 

generate_game(x,6)
        

[[2, 3, 10, 5, 4, 9, 7],
 [3, 8, 2, 9, 7, 6, 5],
 [4, 10, 6, 9, 8, 2, 5],
 [5, 4, 2, 6, 7, 10, 8],
 [6, 9, 3, 2, 4, 7, 10],
 [7, 5, 10, 8, 3, 9, 2],
 [8, 10, 9, 4, 5, 6, 3],
 [9, 3, 2, 6, 7, 4, 8],
 [10, 7, 9, 3, 5, 6, 8]]

In [14]:
class Trainer():
    def __init__(self):
        self.n_population = N_POPULATION
        self.population = self.generateGenes()
        self.currentMaxFitNess = -100000
    
    def generateGenes(self):
        return [Gene(None) for i in range(self.n_population)]
    
    def playWithBot(self):
        for i in range(self.n_population):
            print("______________",i,"___________")
            thisGene = self.population[i]
            thisGene.fit_ness =  Modifiedtestperf("Group 48", thisGene.makePlayerFromGene(), "Bot1", Bot1(),10,10)
            
        
    def playAround(self):
        n_opp_for_each_player = 2
        randomGameAssignment = generate_game(self.population, n_opp_for_each_player)
        for i in range(self.n_population):
            print("______________",i,"___________")
            thisGene = randomGameAssignment[i][0]
#             print(thisGene)
            #ResetFitNess:
            thisGene.fit_ness = 0
            
            score = np.zeros(n_opp_for_each_player)
#             score[0] = thisGene.testPerformance()
#             scoreVsRaise, scoreVsRandom = thisGene.testPerformance()
            for opp_idx in range(n_opp_for_each_player):
                opp_gene = randomGameAssignment[i][1+opp_idx]
                opp_agent = opp_gene.makePlayerFromGene()
                
                this_agent = thisGene.makePlayerFromGene()
                score[opp_idx] = Modifiedtestperf("Group 48", this_agent, "Group 48", opp_agent)
                
            thisGene.fit_ness = np.mean(score)
                
    def train(self):
        for it in range(N_ITER):
            self.playWithBot()
            print("Finish Playing")
            self.expandByCrossOver()
            self.expandByMutation()
            self.reducePopulation()
            
            max_gene = self.population[0]
            min_gene = self.population[-1]
            
            print("Current Max: ", max_gene.fit_ness)
            print("Current Min: ", min_gene.fit_ness)
            print("Eval max gene", max_gene.testPerformance())
            #### Save Best Gene:
            if max_gene.fit_ness > self.currentMaxFitNess:
                self.currentMaxFitNess = max_gene.fit_ness
                max_gene.saveGeneToFile()
                print("Eval new Champion", max_gene.testPerformance())
                
            
    def expandByCrossOver(self):
        for i in range(N_CROSS_OVER):
            twoGeneIdx = np.random.choice(self.n_population, size = 2, replace = False)
            self.population.append(Gene.CrossOver(self.population[twoGeneIdx[0]],self.population[twoGeneIdx[1]]))
        self.n_population += N_CROSS_OVER
        
    def expandByMutation(self):
        for i in range(N_MUTATE):
            geneToMutate = np.random.randint(self.n_population)
            self.population.append(Gene.Mutate(self.population[geneToMutate]))
        self.n_population += N_MUTATE
    
    def reducePopulation(self):
#         print(self.population)
        self.population.sort(key=lambda x:x.fit_ness,reverse =True)
        self.population = self.population[:N_POPULATION]
        self.n_population = N_POPULATION
        
        

In [15]:
trainer = Trainer()

In [16]:
import cPickle
trainer.train()

('______________', 0, '___________')
('Game number: ', 1)
('Game number: ', 2)
('Game number: ', 3)
('Game number: ', 4)
('Game number: ', 5)
('Game number: ', 6)
('Game number: ', 7)
('Game number: ', 8)
('Game number: ', 9)
('Game number: ', 10)
('______________', 1, '___________')
('Game number: ', 1)
('Game number: ', 2)
('Game number: ', 3)
('Game number: ', 4)
('Game number: ', 5)
('Game number: ', 6)
('Game number: ', 7)
('Game number: ', 8)
('Game number: ', 9)
('Game number: ', 10)
('______________', 2, '___________')
('Game number: ', 1)
('Game number: ', 2)
('Game number: ', 3)
('Game number: ', 4)
('Game number: ', 5)
('Game number: ', 6)
('Game number: ', 7)
('Game number: ', 8)
('Game number: ', 9)
('Game number: ', 10)
('______________', 3, '___________')
('Game number: ', 1)
('Game number: ', 2)
('Game number: ', 3)
('Game number: ', 4)
('Game number: ', 5)
('Game number: ', 6)
('Game number: ', 7)
('Game number: ', 8)
('Game number: ', 9)
('Game number: ', 10)
('______

KeyError: 0